여러 사람이 detection됨 -> 운동하는 사람만 남기기

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import csv
sys.path.append('/home/hrai/codes/PoseAdaptor')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT/data/action')

In [81]:
def get_bbox_from_pose2d(pose_2d):
    x1 = int(min(pose_2d[:, 0]))
    x2 = int(max(pose_2d[:, 0]))
    y1 = int(min(pose_2d[:, 1]))
    y2 = int(max(pose_2d[:, 1]))
    area = (x2 - x1) * (y2 - y1)
    #print(x1, y1, x2, y2, x2-x1, y2-y1, area)
    return x1, y1, x2, y2, area

### Check fit3d_hrnet.pkl

In [91]:
data = readpkl('./fit3d_hrnet.pkl')
print(data.keys())

dict_keys(['split', 'annotations'])


In [92]:
print(data['split'].keys())
print(data['split']['train'])
print(data['split']['test'])

dict_keys(['train', 'test'])
['train_s05_50591643_pushup', 'train_s10_50591643_squat', 'train_s11_60457274_pushup', 'train_s11_50591643_squat', 'train_s11_58860488_burpees', 'train_s08_60457274_squat', 'train_s04_65906101_pushup', 'train_s10_60457274_burpees', 'train_s11_60457274_burpees', 'train_s07_58860488_pushup', 'train_s11_65906101_burpees', 'train_s03_65906101_burpees', 'train_s09_60457274_pushup', 'train_s04_65906101_burpees', 'train_s05_58860488_pushup', 'train_s11_60457274_squat', 'train_s03_50591643_pushup', 'train_s10_50591643_pushup', 'train_s04_50591643_pushup', 'train_s07_58860488_burpees', 'train_s05_60457274_burpees', 'train_s10_50591643_burpees', 'train_s08_50591643_burpees', 'train_s10_58860488_burpees', 'train_s08_65906101_squat', 'train_s03_60457274_pushup', 'train_s10_65906101_squat', 'train_s08_65906101_pushup', 'train_s04_60457274_squat', 'train_s10_60457274_pushup', 'train_s07_58860488_squat', 'train_s09_50591643_burpees', 'train_s04_65906101_squat', 'train_s11

In [93]:
num_videos = len(data['annotations'])
print('number of videos : ', len(data['annotations'])) # number of videos
print(data['annotations'][0].keys())
print('video name : ', data['annotations'][0]['frame_dir']) # video name
print('action label : ', data['annotations'][0]['label']) # action label
print('W, H : ', data['annotations'][0]['img_shape']) # W, H
print('number of frames : ', data['annotations'][0]['total_frames']) # number of frames
print('number of people : ', data['annotations'][0]['num_person_raw']) # number of people
print('keypoint : ', data['annotations'][0]['keypoint'].shape) # keypoint (num_person, total_frames, 17, 2)
print('confidence : ', data['annotations'][0]['keypoint_score'].shape) # confidence (num_person, total_frames, 17)

number of videos :  245
dict_keys(['frame_dir', 'label', 'img_shape', 'total_frames', 'num_person_raw', 'keypoint', 'keypoint_score'])
video name :  train_s05_50591643_pushup
action label :  2
W, H :  (900, 900)
number of frames :  794
number of people :  2
keypoint :  (2, 794, 17, 2)
confidence :  (2, 794, 17)


In [94]:
for video in range(num_videos):
    video_name = data['annotations'][video]['frame_dir']
    action_label = data['annotations'][video]['label']
    W, H = data['annotations'][video]['img_shape']
    num_frames = data['annotations'][video]['total_frames']
    num_people = data['annotations'][video]['num_person_raw']
    keypoint = data['annotations'][video]['keypoint']
    confidence = data['annotations'][video]['keypoint_score']
    print(keypoint.shape)
    main_person = None
    max_area = 0
    for frame in range(1, num_frames):
        for person in range(num_people):
            pose_2d = keypoint[person, frame, :, :]
            _, _, _, _, bbox_area = get_bbox_from_pose2d(pose_2d)
            if bbox_area > max_area:
                max_area = bbox_area
                main_person = person
        if main_person is not None:
            break
    print('main person : ', main_person)

(2, 794, 17, 2)
main person :  0
(1, 1481, 17, 2)
main person :  0
(1, 1064, 17, 2)
main person :  0
(2, 1968, 17, 2)
main person :  0
(2, 1343, 17, 2)
main person :  0
(2, 1247, 17, 2)
main person :  0
(1, 920, 17, 2)
main person :  0
(1, 1589, 17, 2)
main person :  0
(1, 1340, 17, 2)
main person :  0
(1, 774, 17, 2)
main person :  0
(1, 1343, 17, 2)
main person :  0
(1, 854, 17, 2)
main person :  0
(1, 696, 17, 2)
main person :  0
(1, 1883, 17, 2)
main person :  0
(2, 796, 17, 2)
main person :  0
(1, 1968, 17, 2)
main person :  0
(2, 965, 17, 2)
main person :  0
(1, 1042, 17, 2)
main person :  0
(2, 918, 17, 2)
main person :  0
(1, 1629, 17, 2)
main person :  0
(1, 996, 17, 2)
main person :  0
(1, 1589, 17, 2)
main person :  0
(2, 868, 17, 2)
main person :  0
(1, 1591, 17, 2)
main person :  0
(1, 1248, 17, 2)
main person :  0
(1, 965, 17, 2)
main person :  0
(2, 1485, 17, 2)
main person :  0
(1, 698, 17, 2)
main person :  0
(1, 1275, 17, 2)
main person :  0
(2, 1042, 17, 2)
main pers

### Check kookmin_hrnet.pkl

In [96]:
data = readpkl('./kookmin_hrnet.pkl')
data.keys()

dict_keys(['split', 'annotations'])

In [97]:
print(data['split'].keys())
print(data['split']['train'])
print(data['split']['test'])

dict_keys(['train', 'test'])
['train_S09_burpee_2', 'train_S07_butterfly_1', 'train_S04_pushup_7', 'train_S04_kneeup_7', 'train_S07_squat_1', 'train_S10_burpee_3', 'train_S01_pushup_3', 'train_S10_burpee_6', 'train_S07_pushup_1', 'train_S04_kneeup_8', 'train_S01_pushup_4', 'train_S10_kneeup_3', 'train_S06_burpee_3', 'train_S10_squat_6', 'train_S09_pushup_2', 'train_S03_butterfly_3', 'train_S01_kneeup_4', 'train_S01_pushup_1', 'train_S03_burpee_5', 'train_S03_kneeup_3', 'train_S06_squat_8', 'train_S09_squat_2', 'train_S04_kneeup_3', 'train_S10_burpee_4', 'train_S04_squat_8', 'train_S04_burpee_7', 'train_S07_butterfly_8', 'train_S07_butterfly_3', 'train_S09_butterfly_8', 'train_S03_kneeup_4', 'train_S09_burpee_6', 'train_S09_kneeup_8', 'train_S03_butterfly_4', 'train_S01_burpee_5', 'train_S06_butterfly_8', 'train_S03_pushup_7', 'train_S06_burpee_1', 'train_S03_burpee_1', 'train_S07_kneeup_6', 'train_S09_burpee_4', 'train_S03_butterfly_1', 'train_S01_butterfly_8', 'train_S10_butterfly_8',

In [103]:
num_videos = len(data['annotations'])
print('number of videos : ', num_videos) # number of videos
print(data['annotations'][0].keys())
print('video name : ', data['annotations'][0]['frame_dir']) # video name
print('action label : ', data['annotations'][0]['label']) # action label
print('W, H : ', data['annotations'][0]['img_shape']) # W, H
print('number of frames : ', data['annotations'][0]['total_frames']) # number of frames
print('number of people : ', data['annotations'][0]['num_person_raw']) # number of people
print('keypoint : ', data['annotations'][0]['keypoint'].shape) # keypoint (num_person, total_frames, 17, 2)
print('confidence : ', data['annotations'][0]['keypoint_score'].shape) # confidence (num_person, total_frames, 17)

number of videos :  398
dict_keys(['frame_dir', 'label', 'img_shape', 'total_frames', 'num_person_raw', 'keypoint', 'keypoint_score'])
video name :  train_S09_burpee_2
action label :  1
W, H :  (1920, 1080)
number of frames :  1051
number of people :  3
keypoint :  (3, 1051, 17, 2)
confidence :  (3, 1051, 17)


### Remove unnecessary persons

In [109]:
modified_data = copy.deepcopy(data)

for video in range(num_videos):
    video_name = data['annotations'][video]['frame_dir']
    action_label = data['annotations'][video]['label']
    W, H = data['annotations'][video]['img_shape']
    num_frames = data['annotations'][video]['total_frames']
    num_people = data['annotations'][video]['num_person_raw']
    keypoint = data['annotations'][video]['keypoint']
    confidence = data['annotations'][video]['keypoint_score']
    print(keypoint.shape)
    main_person = None
    max_area = 0
    for frame in range(1, num_frames):
        for person in range(num_people):
            pose_2d = keypoint[person, frame, :, :]
            _, _, _, _, bbox_area = get_bbox_from_pose2d(pose_2d)
            if bbox_area > max_area:
                max_area = bbox_area
                main_person = person
        if main_person is not None:
            break
    print('main person : ', main_person)
    modified_data['annotations'][video]['num_person_raw'] = 1
    modified_data['annotations'][video]['keypoint'] = np.array([keypoint[main_person, :, :, :]])
    modified_data['annotations'][video]['keypoint_score'] = np.array([confidence[main_person, :, :]])
    
savepkl(modified_data, './kookmin_hrnet_modified.pkl')

(3, 1051, 17, 2)
main person :  0
(1, 555, 17, 2)
main person :  0
(1, 609, 17, 2)
main person :  0
(2, 754, 17, 2)
main person :  0
(3, 695, 17, 2)
main person :  0
(6, 907, 17, 2)
main person :  1
(8, 676, 17, 2)
main person :  2
(3, 907, 17, 2)
main person :  0
(2, 833, 17, 2)
main person :  0
(1, 754, 17, 2)
main person :  0
(7, 676, 17, 2)
main person :  4
(6, 456, 17, 2)
main person :  0
(7, 936, 17, 2)
main person :  0
(2, 508, 17, 2)
main person :  0
(3, 976, 17, 2)
main person :  0
(5, 435, 17, 2)
main person :  1
(7, 615, 17, 2)
main person :  1
(1, 676, 17, 2)
main person :  0
(6, 1396, 17, 2)
main person :  0
(7, 932, 17, 2)
main person :  0
(2, 888, 17, 2)
main person :  0
(3, 804, 17, 2)
main person :  0
(7, 754, 17, 2)
main person :  0
(6, 907, 17, 2)
main person :  0
(1, 806, 17, 2)
main person :  0
(3, 972, 17, 2)
main person :  0
(1, 555, 17, 2)
main person :  0
(7, 555, 17, 2)
main person :  0
(1, 700, 17, 2)
main person :  0
(7, 932, 17, 2)
main person :  2
(3, 1051